# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import time

# Import API key
from api_keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)
        
    # Print the city count to confirm sufficient count
len(cities)

624

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [3]:
#building query url
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"
query_url = f'{url}&appid={api_key}&units={units}&q='

In [7]:
#creating lists with city weather data
temperature = []
humidity = []
cloudiness = []
wind_speed = []
city_list = []
lat_list = []
lng_list = []
country_list = []
date_time_list = []
count = 0

for city in cities:
    
    count += 1
    print(f'City number: {count}. City name: {city}')
    
    try:
        response = requests.get(query_url + city).json()
        temperature.append((response['main']['temp']))
        humidity.append((response['main']['humidity']))
        cloudiness.append((response['clouds']['all']))
        wind_speed.append((response['wind']['speed']))
        lat_list.append((response['coord']['lat']))
        lng_list.append((response['coord']['lon']))
        city_list.append(city)
        country_list.append(response['sys']['country'])
        date_time_list.append(response['dt'])
        
    except:
        print('City not found.')
        
    time.sleep(1)

City number: 1. City name: cape town


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [9]:
#create df from city weather query results
weather_dict = {"City": city_list,
               "Temperature": temperature,
               "%Humidity": humidity,
               "Cloudiness":cloudiness,
               "Wind Speed": wind_speed,
               "Latitude": lat_list,
               "Longitude": lng_list,
               "Country": country_list,
               "Date Time UTC": date_time_list}

weather_data = pd.DataFrame(weather_dict)
weather_data

,City,Temperature,%Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Country,Date Time UTC
0,cape town,13.79,82,75,7.72,-33.9258,18.4232,ZA,1659050360


In [ ]:
#export city data to csv
weather_data.to_csv('weather_data.csv')

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#there are no cities in this df with humidity > 100%
weather_data.loc[weather_data['%Humidity'] > 100]

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
weather_data.head()

In [ ]:
plt.scatter(weather_data['Latitude'], weather_data['Temperature'])
plt.xlabel('Latitude')
plt.ylabel('Temperature (C)')
plt.title('City Temperature vs Latitude')

plt.show()

Explanation: This code is analyzing the relationship between temperature and latitude for the queried cities.

## Latitude vs. Humidity Plot

In [ ]:
plt.scatter(weather_data['Latitude'], weather_data['%Humidity'])
plt.xlabel('Latitude')
plt.ylabel('Humidity (%)')
plt.title('City Humidity vs Latitude')

plt.show()

Explanation: This code is analyzing the relationship between humidity and latitude for the queried cities.

## Latitude vs. Cloudiness Plot

In [ ]:
plt.scatter(weather_data['Latitude'], weather_data['Cloudiness'])
plt.xlabel('Latitude')
plt.ylabel('Cloudiness (%)')
plt.title('City Cloudiness vs Latitude')

plt.show()

Explanation: This code is analyzing the relationship between cloudiness and latitude for the queried cities.

## Latitude vs. Wind Speed Plot

In [ ]:
plt.scatter(weather_data['Latitude'], weather_data['Wind Speed'])
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (mph)')
plt.title('City Wind Speed vs Latitude')

plt.show()

Explanation: This code is analyzing the relationship between wind speed and latitude for the queried cities.

## Linear Regression

In [ ]:
#Creating dataframes for each hemisphere
north_data = weather_data.loc[weather_data['Latitude'] > 0]
south_data = weather_data.loc[weather_data['Latitude'] < 0]

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
plt.scatter(north_data['Latitude'], north_data['Temperature'])
plt.xlabel('Latitude')
plt.ylabel('Temperature (C)')
plt.title('Northern Hemisphere City Temperature vs Latitude')


(slope, intercept, rvalue, pvalue, stderr) = linregress(north_data['Latitude'], north_data['Temperature'])
regress_values = north_data['Latitude'] * slope + intercept
line_eq = f'y = {round(slope,2)}x + {round(intercept, 2)}'

plt.plot(north_data['Latitude'], regress_values, color='red')
plt.annotate(line_eq,(1,5), color='red', fontsize=10)

plt.xlim(0, 80)
plt.ylim(0, 50)

plt.show()
print(f' rsquared = {round(rvalue * rvalue,2)}')

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
plt.scatter(south_data['Latitude'], south_data['Temperature'])
plt.xlabel('Latitude')
plt.ylabel('Temperature (C)')
plt.title('Southern Hemisphere City Temperature vs Latitude')


(slope, intercept, rvalue, pvalue, stderr) = linregress(south_data['Latitude'], south_data['Temperature'])
regress_values = south_data['Latitude'] * slope + intercept
line_eq = f'y = {round(slope,2)}x + {round(intercept, 2)}'

plt.plot(south_data['Latitude'], regress_values, color='red')
plt.annotate(line_eq,(-45,35), color='red', fontsize=10)

plt.xlim(-50, 0)
plt.ylim(0, 40)

plt.show()
print(f' rsquared = {round(rvalue * rvalue,2)}')

The linear regressions are modeling the dependence of temperature on latitude for both the Northern and Southern Hemisphere. The plots show that in the North Hemisphere, temperature decreases as latitude increases. In the Southern Hemisphere, temperature increases as latitude increases.

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
plt.scatter(north_data['Latitude'], north_data['%Humidity'])
plt.xlabel('Latitude')
plt.ylabel('Humidity (%)')
plt.title('Northern Hemisphere City Humidity vs Latitude')


(slope, intercept, rvalue, pvalue, stderr) = linregress(north_data['Latitude'], north_data['%Humidity'])
regress_values = north_data['Latitude'] * slope + intercept
line_eq = f'y = {round(slope,2)}x + {round(intercept, 2)}'

plt.plot(north_data['Latitude'], regress_values, color='red')
plt.annotate(line_eq,(55,5), color='red', fontsize=10)

plt.show()
print(f' rsquared = {round(rvalue * rvalue,2)}')

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
plt.scatter(south_data['Latitude'], south_data['%Humidity'])
plt.xlabel('Latitude')
plt.ylabel('Humidity (%)')
plt.title('Southern Hemisphere City Humidity vs Latitude')


(slope, intercept, rvalue, pvalue, stderr) = linregress(south_data['Latitude'], south_data['%Humidity'])
regress_values = south_data['Latitude'] * slope + intercept
line_eq = f'y = {round(slope,2)}x + {round(intercept, 2)}'

plt.plot(south_data['Latitude'], regress_values, color='red')
plt.annotate(line_eq,(-45,35), color='red', fontsize=10)

plt.show()
print(f' rsquared = {round(rvalue * rvalue,2)}')

The linear regressions are modeling the dependence of humidity on latitude for both the Northern and Southern Hemispheres. The low r squared values provide evidence supporting the claim that humidity is not dependent on latitude for either hemisphere.

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
plt.scatter(north_data['Latitude'], north_data['Cloudiness'])
plt.xlabel('Latitude')
plt.ylabel('Cloudiness (%)')
plt.title('Northern Hemisphere City Cloudiness vs Latitude')


(slope, intercept, rvalue, pvalue, stderr) = linregress(north_data['Latitude'], north_data['Cloudiness'])
regress_values = north_data['Latitude'] * slope + intercept
line_eq = f'y = {round(slope,2)}x + {round(intercept, 2)}'

plt.plot(north_data['Latitude'], regress_values, color='red')
plt.annotate(line_eq,(55,41), color='red', fontsize=12)

plt.show()
print(f' rsquared = {round(rvalue * rvalue,2)}')

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
plt.scatter(south_data['Latitude'], south_data['Cloudiness'])
plt.xlabel('Latitude')
plt.ylabel('Cloudiness (%)')
plt.title('Southern Hemisphere City Cloudiness vs Latitude')


(slope, intercept, rvalue, pvalue, stderr) = linregress(south_data['Latitude'], south_data['Cloudiness'])
regress_values = south_data['Latitude'] * slope + intercept
line_eq = f'y = {round(slope,2)}x + {round(intercept, 2)}'

plt.plot(south_data['Latitude'], regress_values, color='red')
plt.annotate(line_eq,(-45,25
                     ), color='red', fontsize=10)

plt.show()
print(f' rsquared = {round(rvalue * rvalue,2)}')

The linear regressions are modeling the dependence of cloudiness on latitude for both the Northern and Southern Hemispheres. The low r squared values provide evidence supporting the claim that cloudiness is not dependent on latitude for either hemisphere.

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
plt.scatter(north_data['Latitude'], north_data['Wind Speed'])
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (mph)')
plt.title('Northern Hemisphere City Wind Speed vs Latitude')


(slope, intercept, rvalue, pvalue, stderr) = linregress(north_data['Latitude'], north_data['Wind Speed'])
regress_values = north_data['Latitude'] * slope + intercept
line_eq = f'y = {round(slope,2)}x + {round(intercept, 2)}'

plt.plot(north_data['Latitude'], regress_values, color='red')
plt.annotate(line_eq,(55,12), color='red', fontsize=10)

plt.show()
print(f' rsquared = {round(rvalue * rvalue,2)}')

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
plt.scatter(south_data['Latitude'], south_data['Wind Speed'])
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (mph)')
plt.title('Southern Hemisphere City Wind Speed vs Latitude')


(slope, intercept, rvalue, pvalue, stderr) = linregress(south_data['Latitude'], south_data['Wind Speed'])
regress_values = south_data['Latitude'] * slope + intercept
line_eq = f'y = {round(slope,2)}x + {round(intercept, 2)}'

plt.plot(south_data['Latitude'], regress_values, color='red')
plt.annotate(line_eq,(-50,8), color='red', fontsize=15)

plt.show()
print(f' rsquared = {round(rvalue * rvalue,2)}')

The linear regressions are modeling the dependence of wind speed on latitude for both the Northern and Southern Hemispheres. The low r squared values provide evidence supporting the claim that wind speed is not dependent on latitude for either hemisphere.